In [ ]:
%matplotlib inline

# create driver file to move Y33L

In [ ]:
import os
gros = []
tops = []
the_dir = '/home/domain/data/kirill/AB/XOP/md/a5/S/md-qm-transition/prfs'
os.chdir(the_dir)

for dr in os.walk("."): 
    files = dr[2]
    break
print files

for name in files:      
    p = re.findall("(.*)Y(.*)L.*prf*", name)
    abname = p[0][0]
    y = int(p[0][1])


    if os.path.isfile(abname+"-move-y33l.dat") and y == 33 or os.path.isfile(abname+"-move-y37l.dat") and y == 37: continue
    print abname     
    m = pmx.Model(name)
    
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o = [str(i.id)for i in lig.atoms if "O" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    o2 = [str(i.id)for i in lig.atoms if "O2" in i.name][0]
    c8 = [str(i.id)for i in lig.atoms if "C8" in i.name][0]


    pros = m.fetch_residues("TYR")
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL") and y == 33:
            ats = pro.atoms   
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN") and y == 37:
            ats = pro.atoms   

    oh=[str(i.id)for i in ats if "OH" in i.name][0]
    cg=[str(i.id)for i in ats if "CG" in i.name][0]
    cb=[str(i.id)for i in ats if "CB" in i.name][0]
    ca=[str(i.id)for i in ats if "CA" in i.name][0]
    c=[str(i.id)for i in ats if "C" == i.name][0]
    name = name.split("/")[-1]
    f = open(abname+"-move-y"+str(y)+"l.dat", 'w')
    f.write("\
d: DISTANCE ATOMS={p},{oh}\n\
UPPER_WALLS ARG=d AT=0.3 KAPPA=5000\n\
    ".format(p=p,oh=oh))
    f.close()
    


# create qmmm-ready topologies after a moving Y33L

In [1]:
import pmx 
import pmx._pmx as pmxbin
import pmx.ndx 
import operator
import re
import sys, os
import imp
qm_yeah = imp.load_source('qm_yeah', '/home/domain/data/kirill/AB/XOP/md-qmmm-transition/scripts/qm_yeah.py')
from subprocess import Popen, PIPE
from shlex import split
import time
from collections import OrderedDict as odict

In [3]:
imp.PY_SOURCE

1

In [ ]:
def swap_mols(res1,res2,s):
    """
    This function returns a pmx.model, after doing a pairwise swap 
    of the coordiantes of water molecules from res1 to res2 in a pmx.model s. 
    We need it to make the QM index continuous.
    
    res1 = []
    res2 = []
    s = pmx.model
    """ 

    if len(res1) != len(res2):
        print "Residue lists are different"
        return
        
    for k in range(len(res1)):
        m1 = s.residues[res1[k]-1]
        m2 = s.residues[res2[k]-1]
        m1_c=m1.coords()
        m2_c=m2.coords()
        if len(m2_c) != len(m1_c):
            print "Atom lists are different"
            return
        
        for i in range(len(m1_c)):
            #print m1.atoms[i].name,m2.atoms[i].name
            m1.atoms[i].x = m2_c[i]
            m2.atoms[i].x = m1_c[i]
    return s
    
def process_top (topfile,molcount):
    """
    Explicitly adds water to a topology file
    then writes it to disk with a suffix "_swap"
    
    topfile - path to top file
    molcount - int
    """
  
    qmsol = [['         OW', ' SOL      OW       ','      -0.834    16.00000'],
             ['         HW', ' SOL      HW1      ','       0.417     1.00800'],
             ['         HW', ' SOL      HW2      ','       0.417     1.00800']]
     

    topin = open(topfile, 'r')
    topout= open(topfile[0:-4] + '_swap.top', 'w')
    toparr = topin.readlines()
    topin.close()
    found = False # switch for last atom find
    c = 0 # backwards counter for last atom search
    for k,l in enumerate(toparr):        
        # skip to bonds
        if '[ bonds ]' in l:
            # reverse search for atom records
            while not found:
                try: 
                    c+=1
                    atom_number = int(toparr[k-c].split()[0])
                    res = int(toparr[k-c].split()[2])
                    found = True
                except IndexError:
                    continue
                        
            # update the bonds record
            for i in range(molcount):
                res += 1
                for atom in qmsol:
                    atom_number += 1
                    
                    topout.write("%6d %2s %6d %4s %5d %2s\n" % (atom_number,atom[0],res,atom[1],atom_number, atom[2]))
                              
        # update the number of molecules record
        if 'SOL ' in l:
            tmp,nw=l.split()
            topout.write("%s %d \n" % (tmp,int(nw)-molcount))
            print "changing the number of mm waters from " + nw + " to  " + str(int(nw)-molcount)
        else:
            topout.write(l)  
    topout.close()
    return

def mycom(sele):
    """
    Returns center of mass for a list of pmx.atoms

    sele = [<pmx.atom>]
    """
    
    x = sum(map(lambda a: a.x[0], sele))
    y = sum(map(lambda a: a.x[1], sele))
    z = sum(map(lambda a: a.x[2], sele))
    M = sum(map(lambda a: 1, sele))
    x/=M
    y/=M
    z/=M
    return [x,y,z]

def get_sidechain(m,res):  
    """
    Returns a list of sidechain atoms.
    
    m = pmx.model
    res = int
    """
    
    backbone = ['CA','N','H','C','O','HA']
    residue = m.residues[res]
    atoms = residue.atoms
    atoms_backbone = residue.fetch_atoms(backbone)
    sidechain = list(set(atoms) - set(atoms_backbone))
   # print str(res) + " " + m.residues[res].resname + ", atoms: " + str([a.name for a in sidechain])
    return sidechain


def find_close(m, com, resnames, dist, ref = None):
    """
    Neighbour search around a point (com). 
    Suitable names of neighbours are specified in "resnames"
    A residue is considered a neighbour if its center of mass
    is within "dist" of given "com".
    
    m = pmx.model
    com = [x,y,z]
    resnames = [<str>]
    dist = float
    """
    
    close = []
    
    if resnames == 'all': #for finding aminoacids by closest atoms
        for r in m.residues:
            if r.resname not in ['SOL', 'HOH', 'CL', 'NA', 'XQM', 'GLY',"PRO"]:
                check = min([pmxbin.dist(com,at.x) for at in r.atoms])
                if check < dist:
                    close.append(r.id)
    else:    
        for r in m.residues: #for finding water 
            if r.resname == "HOH" or r.resname == "SOL":
                check = pmxbin.dist(com, mycom(r.atoms))
                check_ref = pmxbin.dist(ref, mycom(r.atoms))
                if check < dist and check_ref < 0.8:
                    close.append(r.id)               
    return close
        
def get_atoms_ndx (gro, resi, notqm):
    qmlist=[]
    out = []
    for a in gro:
        if a[0] == resi and not( a[2].strip() in notqm) :
            qmlist.append(a[3])  
    return qmlist



def get_aa_link (gro, resi):
# given and index and a "prepare_qm.GRO" object returns a list of [??? , 'CT', 'HC']
#  CT - HC is the substitution
    list=[]
    for a in gro:
        if a[0] == resi and ( a[2].strip() == 'CA' or a[2].strip() == 'CB' ) :        
            list.append(a[3])

    list.reverse()
    list.extend(['CT','HC'])   
    return list    



def get_shit_done_qm(gro, top, qms, lig):
    """
    Adds linking atoms
    Writes .gro, .top, .ndx with suffix _qm
    
    
    """
    
    qm = qm_yeah.AddLinkingAtoms({})
    qm.gro = qm_yeah.GRO(open(gro))

    qmlist = []
    la_list = []
    backbone = ['CA','N','H','C','O','HA']

    for res in qms:
        qmlist.extend(get_atoms_ndx(qm.gro.atoms,res,backbone))
        la_list.append(get_aa_link (qm.gro.atoms,res))    
    
    qmlist.extend(get_atoms_ndx(qm.gro.atoms,lig,[])) # the ligand

    #print "qm list is: " + str(qmlist)
    #print "la list is: " + str(la_list)
    
    groout = gro[0:-4]+'_qm.gro'
    grofile    = open(gro)
    grooutfile = open(groout,'w')

    
    topout = top[0:-4]+'_qm.top'
    topfile    = open(top)
    topoutfile = open(topout,"w")

    path=gro.split('/')
    qmifile = open('./'.join(path[0:-1]) + 'temp.ndx','w')  

    qm.bonds2break = la_list

    qm.groups = odict()

    qm.groups['System'] = range(1,qm.gro.num + 1)
    qm.groups['Prot'] = range(1,qmlist[-1] + 1)
    qm.groups['QM'] = qmlist
    qm.group = 'QM'

    qm.process_topology(topfile,topoutfile)

    qm.write_index(qmifile)
    qm.add_la_coordinates()

    qm.gro.write(grooutfile)
    
    grofile.close()
    grooutfile.close()
    topfile.close()
    topoutfile.close()
    qmifile.close()
    return

def get_charge(m, qm, lig):
    """
    Returns an integer total charge of residues from model "m",
    listed in "qm" and also the lig charge.
        
    m = pmx.model
    qm = [<int>]
    lig = [<int>]
    """
    
    q = lig
    pos = ['ARG', 'HIP', 'LYS']
    neg = ['ASP', 'GLU',]
    for res in qm:
        if m.residues[res-1].resname in neg:
            q -= 1
        if m.residues[res-1].resname in pos:
            q += 1
    return q

In [ ]:
the_dir = '/home/domain/data/kirill/AB/XOP/md/a17/R/md-qm-transition/prfs/'
os.chdir(the_dir)
for dr in os.walk("."): 
    dirs = dr[1]
    break
dz = [dr for dr in dirs if "pre-qm.gro" in dr]
print dirs

In [ ]:
-

In [ ]:
os.chdir(the_dir + "/" + dirs[0])
gros = [dr for dr in os.listdir(".") if "-qm.gro" in dr and not "-pre-qm" in dr]

In [ ]:
gros

In [ ]:
md = pmx.Model(gros[0])

In [ ]:
subprocess.call([printf "1 | 13 | 14\n!26\nq\n" | make_ndx -f {grob} -o {ndx}])

In [ ]:
nd.ids[-1]

In [ ]:
asd = qms.fetch_atoms('P',how='byname')[0].id

In [ ]:
asd

### search for R-S conversions

In [ ]:
sex = open('/home/domain/data/kirill/AB/XOP/md_noqm/mdR_ncv_rot_pb/test.pdb','r')
P = []
O = []
O1 = []
O2 = []
C8 = []

for line in sex:
    
    if " P " in line: P.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
    if " O " in line: O.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
    if "O1" in line: O1.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
    if "O2" in line: O2.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
    if "C8" in line: C8.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
sex.close()
        

In [ ]:
import numpy as np

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle = np.arccos(np.dot(v1_u, v2_u))
    if np.isnan(angle):
        if (v1_u == v2_u).all():
            return 0.0
        else:
            return np.pi
    return angle   

In [ ]:
result = []
for at in range(len(P)):
    a = np.subtract(O[at],O1[at])
    b = np.subtract(O[at],C8[at])
    x = np.subtract(P[at],O2[at])
    n = np.cross(a,b)
    result.append(angle_between(x,n))

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
x = range(len(result))
plt.plot(x, result)
plt.title('R-S transition')
plt.show()

##### select reation-favorable geometry from pdb file

In [ ]:
files

### system size test

In [ ]:
import os
gro = 'conf.gro'
top = 'topol.top'
os.chdir('/home/domain/data/kirill/AB/XOP/test/ref_reaction/')
LIG = 4 # res number of ligand
lig_charge = 1
R = 0.1 # radius for finding aminoacids
D = 0.5 # radius for finding water around qm sidechains
attacking = 2 # res index of attacking aa
important = []
qm_last = []
while R < 1.0 and len(qm_last) < 3:
    qm =[]

    print "processing " + top[:-4]

    m = pmx.Model(gro)  

    #########################################
    #
    # find aminoacids for qm
    #
    #########################################

    p = m.residues[LIG-1].fetch_atoms(["P"])[0]
    o2= m.residues[LIG-1].fetch_atoms(['O2'])[0]
    o = m.residues[attacking-1].fetch_atoms(["OH"])[0]
    while (len(qm) <= len(qm_last)):
        qm = find_close(m, p.x, 'all', R) # around phosphorous
        qm = find_close(m, o.x, 'all', R) # around tyrosine oxygen
        R += 0.01
    qm_last = list(qm)
    
    print "found  " + str(len(qm)) + " aminoacids: \n" + str(qm) + " in radius: "  + str(R) +"\n"

    #get shit done
    get_shit_done_qm(gro, top, qm, LIG)

    !mv topol_qm.top topol{R}_qm.top
    !mv conf_qm.gro topol{R}_qm.gro


    #########################################
    #
    # get sidechain atoms xyz
    #
    #########################################

    reacting = []

    sidechains = [get_sidechain(m,res-1) for res in qm]  
    for sidechain in sidechains:
        reacting.extend([atom.x for atom in sidechain if "H" not in atom.name]) # add sidechain heavy  xyz
    reacting.extend([atom.x for atom in m.residues[LIG-1].atoms if "H" not in atom.name]) # add ligand heavy xyz
    print "will search for water near " + str(len(reacting)) + " atoms"
    


    #########################################
    #
    # find and add qm water near sidechains, updating ndx's
    #
    #########################################

    topo = top[:-4]+str(R)+"_qm.top"
    grob = top[:-4]+str(R)+"_qm.gro"
    ndx = top[:-4]+str(R)+"_qm.ndx"
    m = pmx.Model(grob)
    
    !printf "1 | 13 | 14\n!24\nq\n" | make_ndx -f {grob} -o {ndx}


    ndx_file = pmx.ndx.IndexFile(ndx)

    # we need to make an index file to get atoms by IDs

    solndx = ndx_file['SOL'].ids  
    waterion_NDX = ndx_file['Water_and_ions'].ids
    protndx = ndx_file['Protein_XOP_XXX'].ids
    not_protndx = ndx_file['!Protein_XOP_XXX'].ids
    posrendx = list(protndx)
    nd = pmx.ndx.IndexFile("temp.ndx")
    qmndx = nd['QM'].ids

    # search for waters near qm system

    nb=set() # to make sure waters are unique
    for xyz in [p.x]:
        nb.update(find_close(m, xyz, 'HOH', D, p.x))
    nb=list(nb)
    print "found " + str(len(nb)) + " water(s)"

    first_water = 0
    for r in m.residues: 
        if r.resname == 'SOL':
            first_water = r.id
            break

    # calculate indicies of waters to become QM
    waters = range(first_water, first_water + len(nb))
    print "waters: " + str(waters)
    print "waters: " + str(nb)
    # move them right at the beginning of all waters to make a continuous index group
    m_sw = swap_mols(nb,waters,m)

    # add qm waters to QM and Protein_XOP index groups
    for i in waters:
        residue = m.residues[i-1]
        for a in residue.atoms:
            qmndx.append(a.id)
            protndx.append(a.id)

    # update the QM-containing groups

    ndx_file.add_group(pmx.ndx.IndexGroup(name='QM',ids=qmndx))
    ndx_file.add_group(pmx.ndx.IndexGroup(name='Protein_XOP_XXX',ids=protndx))
    # delete the QM water from the other index grops

    ndx_file.add_group(pmx.ndx.IndexGroup(name='!Protein_XOP_XXX',ids=not_protndx[3*len(nb):]))
    ndx_file.delete_group('SOL')
    ndx_file.add_group(pmx.ndx.IndexGroup(name='SOL',ids=solndx[3*len(nb):]))
    ndx_file.delete_group('Water_and_ions')
    ndx_file.add_group(pmx.ndx.IndexGroup(name='Water_and_ions',ids=waterion_NDX[3*len(nb):]))


    ndx_file.write(ndx[0:-4]+'_swap.ndx')   
    m_sw.writeGRO(grob[0:-4]+'_swap.gro')
    process_top(topo, len(nb))


    #########################################
    #
    # make posre file
    #
    ######################################### 

    pos = open(grob[0:-4] + str(R) + "_swap.itp",'w')
    pos.write("[ position_restraints ]\n")
    for atom in posrendx:
        if ("H" not in m.atoms[atom-1].name):
            pos.write(str(atom)+" 1 100000 100000 100000\n")
    pos.close()

    #########################################
    #
    # make plumed file
    #
    ######################################### 

    qms = m.residues[attacking-1]
    o = qms.fetch_atoms('OH',how='byname')[0].id
    h = qms.fetch_atoms('HH',how='byname')[0].id
    qms = m.residues[LIG-1]
    p = qms.fetch_atoms('P',how='byname')[0].id
    o2 = qms.fetch_atoms('O2',how='byname')[0].id
    plu = open(grob[0:-4] + str(R) + ".dat",'w')
    plu.write("\
OH: DISTANCE ATOMS={0},{1}\n\
d: DISTANCE ATOMS={0},{2}\n\
h: DISTANCE ATOMS={1},{3}\n\
METAD LABEL=metad ARG=OH HEIGHT=0.5 SIGMA=0.02 PACE=10 FILE=HILLS\n\
UPPER_WALLS ARG=d AT=0.25 KAPPA=10000\n\
UPPER_WALLS ARG=h AT=0.20 KAPPA=10000\n\
PRINT ARG=OH,d,metad.bias STRIDE=10 FILE=COLVAR\n\
    ".format(o,h,p,o2))
    plu.close()


    #########################################
    #
    # make total qm charge file
    #
    #########################################    

    fl = open(top[:-4] + str(R)+".q", "w")
    fl.write(str(get_charge(m,qm,lig_charge)))
    print("charge "+str(get_charge(m,qm,lig_charge)))
    fl.close()
    files= [topo[0:-4]+'_swap.top',
            grob[0:-4]+'_swap.gro',
            ndx[0:-4]+'_swap.ndx',
            grob[0:-4] + str(R) + ".dat",
            top[:-4] + str(R)+".q",
            grob[0:-4] + str(R) + "_swap.itp"]
    !mkdir {R}
    !mkdir {R}-2
    !mv {files[0]} {R}/topol.top
    !mv {files[1]} {R}/conf.gro
    !mv {files[2]} {R}/index.ndx
    !mv {files[3]} {R}/plumed.dat
    !mv {files[4]} {R}/charge.q
    !mv {files[5]} {R}/posre.itp
    !cp -r {R}/* {R}-2
    
    break

In [ ]:
asd=m.residues[307]

In [ ]:
[a.x for a in m.residues[LIG-1].atoms if "H" not in a.name]

In [ ]:
!mkdir
